In [1]:
from mne_bids import BIDSPath, write_raw_bids, get_anonymization_daysback
import random
import pandas as pd
import numpy as np
import itertools
import mne
import os
import sys
import re
from util.io.add_stream_to_event_tags import *
from util.io.remap_aux_labels import *
from util.io.iter_raw_paths import *

In [3]:
RAW_DIR = '../data/raw/'
BIDS_DIR = '../data/bids/'
MAPS_DIR = '../data/captrak/'

In [11]:
for fpath, sub, task, run in iter_raw_paths(RAW_DIR):
    print(fpath, sub, task, run)

    # load data with MNE function for your file format
    full_fpath = os.path.join(RAW_DIR, fpath)
    print(full_fpath)
    raw = mne.io.read_raw_brainvision(full_fpath)
    raw.load_data()
    
    # Rename channels according to aux-label-remapping.csv
    print("Renaming aux channels")
    raw, remap_dict = remap_aux_labels(sub, raw, 'aux-label-remapping.csv') # First number in tag is left channel
    raw.rename_channels(remap_dict)
    raw.set_channel_types({'Left': 'stim', 'Right': 'stim'})

    # add some info BIDS will want
    print("Add line_freq to raw.info")
    raw.info['line_freq'] = 60 # the power line frequency in the building we collected in

    # map channel numbers to channel names
    print("Map channel numbers to channel names")
    print(f"Original channel names: {raw.ch_names}")
    if int(sub) < 29:
        map_fp = '../data/captrak/pitch_tracking_64_at_FCZ.csv'
        mapping_table = pd.read_csv(map_fp)
        mapping = {mapping_table.number[i]: mapping_table.name[i] for i in range(len(mapping_table))}
        raw.rename_channels(mapping)
    raw.add_reference_channels(ref_channels = ['Cz'])
    print(f"Channel names: {raw.ch_names}")
    
    # Double check that eog channels are labeled as eeg
    raw.set_channel_types({'leog': 'eeg', 'reog': 'eeg'})
    print(f"Channel types: {raw.get_channel_types()}")
    
    # Checks
    n_channels = len(raw.ch_names)
    print(f"Number of channels: {n_channels}")
    if n_channels != 66:
        sys.exit(f"Incorrect number of channels, there should be 66 (2 aux incl) channels, instead there are {n_chans} channels")

    # Map channels to their coordinates
    print("Map channels to their captrak coordinates")
    captrak_found = False
    captrak_sub = sub

    while not captrak_found:
        print("Looking for captrak file")
        captrak_path = MAPS_DIR + 'sub-' + str(captrak_sub) + '.bvct'
        print(captrak_path)
        if os.path.isfile(captrak_path):
            print(f"Using captrak file from {captrak_sub}")
            dig = mne.channels.read_dig_captrak(captrak_path)
            raw.set_montage(dig, on_missing = 'warn')
            captrak_found = True
        else:
            captrak_sub = random.randint(3, 30)

    # Extract events from raw file
    print("Set annotations")
    events, event_ids = mne.events_from_annotations(raw)

    # Add stream to event tags
    events = add_stream_to_event_tags(events, sub)

    # Drop meaningless event name
    events = np.array(events)
    events = events[events[:,2] != event_ids['New Segment/'], :]

    # Rename events to their stimulus pitch
    annot = mne.annotations_from_events(events, sfreq = raw.info['sfreq'])
    raw.set_annotations(annot)

    # Get range of dates the BIDS specification will accept
    daysback_min, daysback_max = get_anonymization_daysback(raw)

    # Write data into BIDS directory, while anonymizing
    print("Write data into BIDS directory")
    bids_path = BIDSPath(
            run = run,
            subject = sub,
            task = task,
            datatype = 'eeg',
            root = BIDS_DIR
    )

    write_raw_bids(
        raw,
        bids_path = bids_path,
        allow_preload = True, # whether to load full dataset into memory when copying
        format = 'BrainVision', # format to save to
        anonymize = dict(daysback = daysback_min), # shift dates by daysback
        overwrite = True,
    )

    # Check if conversion was successful and .vhdr file was written
    vhdr_path = str(bids_path)
    print(f".vhdr file written? {os.path.exists(vhdr_path)}")
    print("Done :-)")

sub-24.vhdr
sub-24.vhdr 24 dichotic 1
../data/raw/sub-24.vhdr
Extracting parameters from ../data/raw/sub-24.vhdr...
Setting channel info structure...
Reading 0 ... 17303499  =      0.000 ...  3460.700 secs...
Renaming aux channels
Add line_freq to raw.info
Map channel numbers to channel names
Original channel names: ['Ch1', 'Ch2', 'Ch3', 'Ch4', 'Ch5', 'Ch6', 'Ch7', 'Ch8', 'Ch9', 'Ch10', 'Ch11', 'Ch12', 'Ch13', 'Ch14', 'Ch15', 'Ch16', 'Ch17', 'Ch18', 'Ch19', 'Ch20', 'Ch21', 'Ch22', 'Ch23', 'Ch25', 'Ch26', 'Ch27', 'Ch28', 'Ch29', 'Ch30', 'Ch31', 'Ch32', 'Ch33', 'Ch34', 'Ch35', 'Ch36', 'Ch37', 'Ch38', 'Ch39', 'Ch40', 'Ch41', 'Ch42', 'Ch43', 'Ch44', 'Ch45', 'Ch46', 'Ch47', 'Ch48', 'Ch49', 'Ch50', 'Ch51', 'Ch52', 'Ch53', 'Ch54', 'Ch55', 'Ch56', 'Ch57', 'Ch58', 'Ch59', 'Ch60', 'Ch61', 'Ch62', 'Ch63', 'Ch64', 'Left', 'Right']
Location for this channel is unknown; consider calling set_montage() again if needed.
Channel names: ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'C

/tmp/ipykernel_19314/3980103411.py:53: RuntimeWarning: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The channels missing from the montage are:

['leog', 'reog'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw.set_montage(dig, on_missing = 'warn')


../data/logs/sub-24_blk-2.log
../data/logs/sub-24_blk-4.log
../data/logs/sub-24_blk-0.log
../data/logs/sub-24_blk-1.log
../data/logs/sub-24_blk-3.log
Successfully matched marks and tags :-)
Write data into BIDS directory
Writing '../data/bids/participants.tsv'...
Writing '../data/bids/participants.json'...
Writing '../data/bids/sub-24/eeg/sub-24_space-CapTrak_electrodes.tsv'...
Writing '../data/bids/sub-24/eeg/sub-24_space-CapTrak_coordsystem.json'...
Used Annotations descriptions: ['111', '112', '113', '121', '122', '123', '131', '132', '133', '211', '212', '213', '221', '222', '223', '231', '232', '233']
Writing '../data/bids/sub-24/eeg/sub-24_task-dichotic_run-1_events.tsv'...
Writing '../data/bids/dataset_description.json'...
Writing '../data/bids/sub-24/eeg/sub-24_task-dichotic_run-1_eeg.json'...
Writing '../data/bids/sub-24/eeg/sub-24_task-dichotic_run-1_channels.tsv'...
Copying data files to sub-24_task-dichotic_run-1_eeg.vhdr


/tmp/ipykernel_19314/3980103411.py:86: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(


Writing '../data/bids/sub-24/sub-24_scans.tsv'...
Wrote ../data/bids/sub-24/sub-24_scans.tsv entry with eeg/sub-24_task-dichotic_run-1_eeg.vhdr.
.vhdr file written? True
Done :-)
sub-40.vhdr
sub-40.vhdr 40 dichotic 1
../data/raw/sub-40.vhdr
Extracting parameters from ../data/raw/sub-40.vhdr...
Setting channel info structure...


...

/tmp/ipykernel_19314/3980103411.py:32: RuntimeWarning: The unit for channel(s) leog, reog has changed from NA to V.
  raw.set_channel_types({'leog': 'eeg', 'reog': 'eeg'})
/tmp/ipykernel_19314/3980103411.py:53: RuntimeWarning: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The channels missing from the montage are:

['leog', 'reog'].

Consider using inst.rename_channels to match the montage nomenclature, or inst.set_channel_types if these are not EEG channels, or use the on_missing parameter if the channel positions are allowed to be unknown in your analyses.
  raw.set_montage(dig, on_missing = 'warn')


../data/logs/sub-40_blk-2.log
../data/logs/sub-40_blk-0.log
../data/logs/sub-40_blk-1.log
Successfully matched marks and tags :-)
Write data into BIDS directory
Writing '../data/bids/participants.tsv'...
Writing '../data/bids/participants.json'...
Writing '../data/bids/sub-40/eeg/sub-40_space-CapTrak_electrodes.tsv'...
Writing '../data/bids/sub-40/eeg/sub-40_space-CapTrak_coordsystem.json'...
Used Annotations descriptions: ['111', '112', '113', '121', '122', '123', '131', '132', '133', '211', '212', '213', '221', '222', '223', '231', '232', '233']
Writing '../data/bids/sub-40/eeg/sub-40_task-dichotic_run-1_events.tsv'...
Writing '../data/bids/dataset_description.json'...
Writing '../data/bids/sub-40/eeg/sub-40_task-dichotic_run-1_eeg.json'...
Writing '../data/bids/sub-40/eeg/sub-40_task-dichotic_run-1_channels.tsv'...
Copying data files to sub-40_task-dichotic_run-1_eeg.vhdr


/tmp/ipykernel_19314/3980103411.py:86: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(


Writing '../data/bids/sub-40/sub-40_scans.tsv'...
Wrote ../data/bids/sub-40/sub-40_scans.tsv entry with eeg/sub-40_task-dichotic_run-1_eeg.vhdr.
.vhdr file written? True
Done :-)
sub-1.vhdr
sub-1.vhdr 1 dichotic 1
../data/raw/sub-1.vhdr
Extracting parameters from ../data/raw/sub-1.vhdr...
Setting channel info structure...
Reading 0 ... 3916749  =      0.000 ...   783.350 secs...
Renaming aux channels


ValueError: New channel names are not unique, renaming failed

In [12]:
raw.ch_names

['Ch1',
 'Ch2',
 'Ch3',
 'Ch4',
 'Ch5',
 'Ch6',
 'Ch7',
 'Ch8',
 'Ch9',
 'Ch10',
 'Ch11',
 'Ch12',
 'Ch13',
 'Ch14',
 'Ch15',
 'Ch16',
 'Ch17',
 'Ch18',
 'Ch19',
 'Ch20',
 'Ch21',
 'Ch22',
 'Ch23',
 'Ch25',
 'Ch26',
 'Ch27',
 'Ch28',
 'Ch29',
 'Ch30',
 'Ch31',
 'Ch32',
 'Ch33',
 'Ch34',
 'Ch35',
 'Ch36',
 'Ch37',
 'Ch38',
 'Ch39',
 'Ch40',
 'Ch41',
 'Ch42',
 'Ch43',
 'Ch44',
 'Ch45',
 'Ch46',
 'Ch47',
 'Ch48',
 'Ch49',
 'Ch50',
 'Ch51',
 'Ch52',
 'Ch53',
 'Ch54',
 'Ch55',
 'Ch56',
 'Ch57',
 'Ch58',
 'Ch59',
 'Ch60',
 'Ch61',
 'Ch62',
 'Ch63',
 'Ch64',
 'Left',
 'Right']